In [2]:
search_for = 135
start_from = 0
threads = 10
thread = 4
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.014051675796508789
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 175
all cases: 723027151
type: [1, 1, 1, 1, 135] 135
cases of this type: 332150625
100000 34.05489087079056
200000 64.03110024658659
300000 79.86864656620702
400000 79.71242112236219
500000 79.02450553860972
600000 80.43398371120645
700000 76.56953840248691
800000 77.33650135705554
900000 71.93084747931856
1000000 76.64784047838523
1100000 67.56488088853662
1200000 64.65408786912715
1300000 25.88011566239834
1400000 70.62588980631863
1500000 72.55024591030688
1600000 74.41461256831619
1700000 75.45918651174635
1800000 69.41158256816209
1900000 75.06104141020747
2000000 74.84963449827777
2100000 71.08273443653341
2200000 74.75459813592414
2300000 71.01366750620032
2400000 72.82155612624855
2500000 21.95144865119884
2600000 69.41134316023003
2700000 89.64945881085545
2800000 88.47344579854783
2900000 87.18078807067566
3000000 90.78152799921968


30100000 73.75384274143735
30200000 70.86453970973885
30300000 69.64304246065093
30400000 71.80674137179881
30500000 68.5820328396388
30600000 69.25071863359277
30700000 68.23074340513594
30800000 14.927561417753543
30900000 70.12575076440585
31000000 66.75399906796179
31100000 67.34173516345525
31200000 71.41153655273001
31300000 69.22142714920619
31400000 73.8055923205026
31500000 74.82169827943667
31600000 73.08436830738981
31700000 71.83878287660556
31800000 26.491414560258296
31900000 77.66690193772105
32000000 72.40645400151573
32100000 78.7289969865027
32200000 69.93638323919136
32300000 31.034629656255046
32400000 75.54605689656137
32500000 73.10761349815729
32600000 73.28090644746123
32700000 72.78210751994644
32800000 76.21700571472361
32900000 74.32030698750057
33000000 70.3049203632593
33100000 67.6411893694809
33200000 53.653487831148105
33300000 38.861690940959946
33400000 66.01118835523528
33500000 68.01400980943556
33600000 72.40627292040014
33700000 72.36747613528085
3

60600000 61.98044722018341
60700000 61.59149561997457
60800000 63.14499907540167
60900000 64.21509259668034
61000000 66.6740578241177
61100000 32.18969521706985
61200000 68.8887670995867
61300000 69.80808839288346
61400000 69.47379357314868
61500000 69.42747746252829
61600000 71.27104102352183
61700000 71.86903288950747
61800000 70.41866826376601
61900000 75.01138126666882
62000000 36.192259681453585
62100000 61.28227107457765
62200000 68.55189177474247
62300000 68.98688457833539
62400000 69.19110597248981
62500000 65.04974015291282
62600000 65.12013877016318
62700000 63.30188839555691
62800000 20.472313775700986
62900000 65.78852316675884
63000000 64.08158405319354
63100000 64.6409865080147
63200000 64.5195308621673
63300000 65.37707207553302
63400000 68.69321910321747
63500000 49.55937120602619
63600000 48.68621092208106
63700000 72.17376731745071
63800000 71.96178770809975
63900000 72.77523070530015
64000000 71.68703514643491
64100000 71.34802866051488
64200000 70.85591096671088
643

90900000 67.54761860119157
91000000 62.41029436264274
91100000 63.60016774922571
91200000 66.49395423350514
91300000 67.42662493907257
91400000 69.36455896073683
91500000 63.102692292405926
91600000 63.330918891312116
91700000 52.2169034289234
91800000 32.358927219730866
91900000 57.785902162939
92000000 58.82897906713997
92100000 61.87450732617355
92200000 57.95703945154695
92300000 19.504492180006274
92400000 58.26117135807482
92500000 60.79242214483034
92600000 57.427270691483805
92700000 59.4036795403125
92800000 43.5992611167926
92900000 39.22837574481136
93000000 62.62944948333791
93100000 60.95531896804456
93200000 60.43159947686505
93300000 63.733883453194444
93400000 62.19248476296898
93500000 64.43514692253021
93600000 66.82918731987414
93700000 62.607171185693126
93800000 69.43439279614402
93900000 67.29292540140305
94000000 60.49775421632105
94100000 63.72088744598391
94200000 37.71305270455454
94300000 42.27309516540485
94400000 56.85044902558416
94500000 57.13859218285203

120400000 67.12549925799537
120500000 62.47675638731337
120600000 64.41282036036947
120700000 63.31306145145548
120800000 68.23380491321753
120900000 65.20764350278218
121000000 64.02500781145031
121100000 64.53812167795725
121200000 60.812903830846615
121300000 58.42290448878685
121400000 61.43043057863748
121500000 23.47788729403511
121600000 59.172169168318256
121700000 60.76923761823029
121800000 32.04162161895732
121900000 47.133548817665215
122000000 61.765980887860685
122100000 49.41318719246731
122200000 36.2350229122482
122300000 58.8059989091761
122400000 62.468559977343006
122500000 59.84459644154182
122600000 61.70652859942478
122700000 61.54146010575371
122800000 65.48296040223245
122900000 68.0169663585273
123000000 67.50728978783917
123100000 64.55368009126563
123200000 62.72427160543471
123300000 62.891421548352035
123400000 61.193650679227225
123500000 58.8450575559062
123600000 62.52409720716236
123700000 58.06225523078934
123800000 62.58547129423316
123900000 59.6578

...

639700000 2.0579227390943484
639800000 1.8026747429879473
639900000 1.8126514693829479
640000000 1.749873284296703
640100000 1.845977903602077
640200000 1.8797999568612318
640300000 1.9009190913529097
640400000 1.8032007371823298
640500000 1.7695319287138562
640600000 1.7745657001658262
640700000 1.74357209850297
640800000 1.7301209993037712
640900000 1.8605303648083389
641000000 2.0484652715500795
641100000 1.913406866053334
641200000 1.7235612078308584
641300000 1.7002498551482659
641400000 1.6829021822508103
641500000 1.615758693759853
641600000 1.670866560169404
641700000 1.7770181636429596
641800000 1.9012347798111857
641900000 1.9550139262598636
642000000 1.8368392131993114
642100000 1.6870868711102267
642200000 1.5794187871786707
642300000 1.5277733003679692
642400000 1.5636168906324586
642500000 1.6176658156736672
642600000 1.7387074411402226
642700000 1.8738889863332273
642800000 1.7622249996286035
642900000 1.7275015268131155
643000000 1.62712051336782
643100000 1.51016319130

667400000 2.277468839137529
667500000 2.2554506056384267
667600000 2.2166586640024084
667700000 2.311497518224752
best so far: 0
type: [1, 1, 5, 3, 9] 135
cases of this type: 4428675
667800000 1.5688285805918343
667900000 0.9874891028276265
668000000 0.9619736075285957
668100000 0.999225374421274
668200000 1.0030726401610492
668300000 0.9663434641359575
668400000 0.9504619858216087
668500000 0.9410750572310667
668600000 0.9378263160179516
668700000 0.9846110494342963
668800000 0.9776327739839316
668900000 0.9252009562882257
669000000 0.8974427457827706
669100000 0.9270768735554596
669200000 0.9535856841309945
669300000 1.0158962769644984
669400000 1.0009259017443697
669500000 0.9066353110749259
669600000 0.9673675802497963
669700000 0.945715761277765
669800000 1.1378678214566038
669900000 1.1761151924409285
670000000 1.0919638249392292
670100000 1.0502102618787124
670200000 1.0582688183618036
670300000 1.1376436248609383
670400000 1.1432917939220224
670500000 1.07639493860909
670600000

692000000 1.3066309624069923
692100000 1.7802981075208055
692200000 1.7079717940972445
692300000 1.2160431974654693
692400000 2.208779733325653
692500000 1.4955164327104509
692600000 1.319536466693969
692700000 2.1489061131336245
692800000 1.2108121962348322
692900000 1.6126347733422148
693000000 1.777540206423919
693100000 1.4891758635348913
693200000 1.7862224403522489
693300000 1.6141927926511144
693400000 1.3271963568279068
693500000 1.8771368108009536
693600000 1.2869780757249039
693700000 1.3808580013265925
693800000 1.7062559061432865
693900000 1.3876454188634555
694000000 1.5170044386143324
694100000 1.653077164762546
694200000 1.3803852058928385
694300000 1.6860872685825188
694400000 1.6533207629848201
694500000 1.4517862191744102
694600000 1.3419163649476489
694700000 1.6001532988744405
694800000 1.7668393116311152
694900000 1.4856712088499906
695000000 1.5019253234687535
695100000 1.5901196153012847
695200000 1.5318180102004608
695300000 1.5344618161124324
695400000 1.259025

713200000 0.10277933344965179
best so far: 0
type: [1, 9, 1, 15, 1] 135
cases of this type: 30375
best so far: 0
type: [1, 9, 3, 1, 5] 135
cases of this type: 50625
best so far: 0
type: [1, 9, 3, 5, 1] 135
cases of this type: 10125
713300000 0.08781348826065462
best so far: 0
type: [1, 9, 5, 1, 3] 135
cases of this type: 18225
best so far: 0
type: [1, 9, 5, 3, 1] 135
cases of this type: 6075
best so far: 0
type: [1, 9, 15, 1, 1] 135
cases of this type: 2025
best so far: 0
type: [1, 15, 1, 1, 9] 135
cases of this type: 98415
713400000 0.1031667421256582
best so far: 0
type: [1, 15, 1, 3, 3] 135
cases of this type: 32805
best so far: 0
type: [1, 15, 1, 9, 1] 135
cases of this type: 10935
best so far: 0
type: [1, 15, 3, 1, 3] 135
cases of this type: 10935
best so far: 0
type: [1, 15, 3, 3, 1] 135
cases of this type: 3645
best so far: 0
type: [1, 15, 9, 1, 1] 135
cases of this type: 1215
best so far: 0
type: [1, 27, 1, 1, 5] 135
cases of this type: 16875
713500000 0.10485443241181176
best 

best so far: 0
type: [9, 1, 1, 3, 5] 135
cases of this type: 16875
best so far: 0
type: [9, 1, 1, 5, 3] 135
cases of this type: 10125
best so far: 0
type: [9, 1, 1, 15, 1] 135
cases of this type: 3375
723000000 0.0003374052847630448
best so far: 0
type: [9, 1, 3, 1, 5] 135
cases of this type: 5625
best so far: 0
type: [9, 1, 3, 5, 1] 135
cases of this type: 1125
best so far: 0
type: [9, 1, 5, 1, 3] 135
cases of this type: 2025
best so far: 0
type: [9, 1, 5, 3, 1] 135
cases of this type: 675
best so far: 0
type: [9, 1, 15, 1, 1] 135
cases of this type: 225
best so far: 0
type: [9, 3, 1, 1, 5] 135
cases of this type: 1875
best so far: 0
type: [9, 3, 1, 5, 1] 135
cases of this type: 375
best so far: 0
type: [9, 3, 5, 1, 1] 135
cases of this type: 75
best so far: 0
type: [9, 5, 1, 1, 3] 135
cases of this type: 405
best so far: 0
type: [9, 5, 1, 3, 1] 135
cases of this type: 135
best so far: 0
type: [9, 5, 3, 1, 1] 135
cases of this type: 45
best so far: 0
type: [9, 15, 1, 1, 1] 135
cases o